In [1]:
# https://towardsdatascience.com/custom-named-entity-recognition-using-spacy-7140ebbb3718
# https://spacy.io/usage/training#api

In [3]:
import spacy
from spacy import displacy
from spacy.training import Example
from spacy.lang.en import English
import random
import json

# labeled_data_path = r"/home/zhenyuan/AttacKG/NLP/Doccano/admin.jsonl"
labeled_data_path = r"/home/zhenyuan/AttacKG/NLP/Doccano/2021-6-1.jsonl"
labeled_data = []
with open(labeled_data_path, "r") as read_file:
    for line in read_file:
        data = json.loads(line)
        labeled_data.append(data)

print('---Read Labeled Data(%d)!---' % len(labeled_data))

# self.nlp.initialize(lambda: self.spacy_data)
# split training and testing set
# training_set = spacy_data[0:19]
# testing_set = spacy_data[20:-1]

---Read Labeled Data(505)!---


In [6]:
# nlp = spacy.blank("en")
# nlp = spacy.load("en_core_web_sm") # python -m spacy download en_core_web_sm
model_location = None
# model_location = "/home/zhenyuan/AttacKG/NLP/cti.model"
# ner_labels = ["NetLoc", "APTFamily", "ExeFile", "ScriptsFile", "DocumentFile", "E-mail", "Registry", "File", "Vulnerability", "C2C", "SensInfo", "Service"]
ner_labels = ["FilePath", "NetLoc", "FileName", "Vulnerability", "Registry", "Attacker", "ExeFile", "DocFIle", "Service"]

if model_location is None:
    nlp = spacy.blank('en')
    print("---Created Blank 'en' Model!---")
else:
    nlp = spacy.load(model_location)
    print("---Load Model: %s!---" % model_location)

if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
else:
    ner = nlp.get_pipe("ner")
print("---Add Pipe 'ner'!---")

for label in ner_labels:
    ner.add_label(label)

if model_location is None:
    optimizer = nlp.begin_training()
else:
    optimizer = ner.create_optimizer()
print("---Created Optimizer!---")

---Created Blank 'en' Model!---
---Add Pipe 'ner'!---
---Created Optimizer!---


In [8]:
# Data format converting
spacy_data = []
for entry in labeled_data:
    entities = []
    for e in entry['label']:
        entities.append((e[0], e[1], e[2]))
    try:
        spacy_data.append(Example.from_dict(nlp.make_doc(entry['data']), {"entities": entities}))
    except:
        print("Wrong format: %s!" % entry['data'])
print(spacy_data)

Wrong format: BBSRAT has been loaded through DLL side-loading of a legitimate Citrix executable that is set to persist through the Registry Run key location HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Run\ssonsvr.exe.
!
Wrong format: If establishing persistence by installation as a new service fails, one variant of Elise establishes persistence for the created .exe file by setting the following Registry key: HKCU\Software\Microsoft\Windows\CurrentVersion\Run\svchost : %APPDATA%\Microsoft\Network\svchost.exe. Other variants have set the following Registry keys for persistence: HKCU\Software\Microsoft\Windows\CurrentVersion\Run\imejp : [self] and HKCU\Software\Microsoft\Windows\CurrentVersion\Run\IAStorD.
!
Wrong format: Silence has used HKCU\Software\Microsoft\Windows\CurrentVersion\Run, HKLM\Software\Microsoft\Windows\CurrentVersion\Run, and the Startup folder to establish persistence. 
!
Wrong format: If a victim meets certain criteria, T9000 uses the AppInit_DLL functionality to a

/home/zhenyuan/anaconda3/envs/cti/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "Turla has used the VPS infrastructure of compromis..." with entities "[(0, 6, 'Attacker'), (19, 23, 'NetLoc')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/zhenyuan/anaconda3/envs/cti/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "PROMETHIUM has created self-signed digital certifi..." with entities "[(0, 11, 'Attacker'), (67, 83, 'NetLoc'), (43, 56,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/zhenyuan/anaconda3/envs/cti/lib/python3.8/site-packages/spacy/training/iob_utils.py

In [11]:
len(spacy_data)

493

In [12]:
# Start training
print("---Start Training!---")
new_model_location = "/home/zhenyuan/AttacKG/NLP/new_cti.model"

# Loop
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):
    for itn in range(4):
        random.shuffle(spacy_data)
        losses = ()

        # Batch the examples
        for batch in spacy.util.minibatch(spacy_data, size=2):
            # Update the model
            nlp.update(batch, sgd=optimizer) #, drop=0.35, losses=losses
            # print('Losses', losses)

nlp.to_disk(new_model_location)
print("---Save Model to %s!---" % new_model_location)

---Start Training!---
---Save Model to None!---


In [13]:
sample = "APT3 has used PowerShell on victim systems to download and run payloads after exploitation."

#def test_model(sample = sample):
doc = nlp(sample)
displacy.render(doc, style='ent')

/home/zhenyuan/anaconda3/envs/cti/lib/python3.8/site-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [61]:
sample = "MCMD can use Registry Run Keys for persistence."
nlp = spacy.load(new_model_location)
# nlp = spacy.load("en_core_web_sm")
doc = nlp(sample)
displacy.render(doc, style = 'ent')

In [6]:
import spacy
from spacy.pipeline import EntityRuler

nlp = spacy.blank('en')
doc = nlp("APT3 has used PowerShell on victim systems to download and run payloads after exploitation.")
print([(ent.text, ent.label_) for ent in doc.ents])

# https://stackoverflow.com/questions/57667710/using-regex-for-phrase-pattern-in-entityruler
# patterns = [{"label": "ExeFile", "pattern": "payloads"}]
patterns = [{"label": "ExeFile", "pattern":[{"TEXT": {"REGEX": "payloads"}}]}]

config = {
   "phrase_matcher_attr": None,
   "validate": True,
   "overwrite_ents": False,
   "ent_id_sep": "||",
}

ruler = nlp.add_pipe("entity_ruler", config=config)
ruler.add_patterns(patterns)

doc = nlp("APT3 has used PowerShell on victim systems to download and run payloads after exploitation.")
print([(ent.text, ent.label_) for ent in doc.ents])

[]
[('payloads', 'ExeFile')]
